In [2]:
import pandas as pd


In [3]:
data1 = pd.read_csv('data1.csv', sep=';')
data2 = pd.read_csv('data2.csv', sep=';')
data3 = pd.read_csv('data3.csv', sep=';')

In [4]:
print(data1.head())
print(data2.head())
print(data3.head())

    ID     RegDate RegCountry RegSource
0  721  2016-01-01          B        SE
1  722  2016-01-01          A        SE
2  723  2016-01-01          A        SE
3  724  2016-01-01          A        SE
4  841  2016-01-01          A        SE
    ID     DepDate               Sum PaymInstr
0  620  2016-01-03  3,06849682010272     Other
1  620  2016-01-03  3,94521019727492     Other
2  620  2016-01-03  4,20822421042658     Other
3  620  2016-01-02  2,63014013151662     Other
4  620  2016-01-02  1,75342675434441     Other
       ID    GameDate  GameHour             SumRevA            SumRevB
0    7315  2016-05-24         7  0,0191815856777494                  0
1  154563  2016-12-23        20    17,3337595907928  -15,3388746803069
2   37537  2016-09-19        18  0,0191815856777494                  0
3    4629  2016-02-25        18                   0                  0
4    4583  2016-02-12         3    9,64194373401535                  0


In [5]:
data1['RegDate'] = pd.to_datetime(data1['RegDate'])
data2['DepDate'] = pd.to_datetime(data2['DepDate'])
data3['GameDate'] = pd.to_datetime(data3['GameDate'])

data2['Sum'] = data2['Sum'].apply(lambda x: float(x.replace(',', '.')))
data3['SumRevA'] = data3['SumRevA'].apply(lambda x: float(x.replace(',', '.')))
data3['SumRevB'] = data3['SumRevB'].apply(lambda x: float(x.replace(',', '.')))

In [6]:
data2['DepMonth'] = data2['DepDate'].dt.strftime('%Y-%m')
data1['RegMonth'] = data1['RegDate'].dt.strftime('%Y-%m')


In [30]:
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376902 entries, 0 to 376901
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   ID          376902 non-null  int64         
 1   RegDate     376902 non-null  datetime64[ns]
 2   RegCountry  376902 non-null  object        
 3   RegSource   376902 non-null  object        
 4   RegMonth    376902 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 14.4+ MB
